In [ ]:
import os

import geopandas as gpd
import pandas as pd

from rasterstats import zonal_stats

import fct_misc
from fct_stats import pca_procedure

In [ ]:
DATA_PATH='/mnt/data-01/gsalamin/deperissement-hetre/'

NORTH_NDVI=os.path.join(DATA_PATH, 'processed/NDVI/*.tif')
SOUTH_NDVI=os.path.join(DATA_PATH, 'processed/NDVI/*.tif')

NORTH_MASK=os.path.join(DATA_PATH, 'processed/CHM/CHM_binary_beurnevesin.tif')
SOUTH_MASK=os.path.join(DATA_PATH, 'processed/CHM/CHM_binary_miecourt.tif')

AOI_GT=os.path.join(DATA_PATH, 'initial/AOI/AOI_tiles/AOI_GT.shp')

BEECHES_POLYGONS=os.path.join(DATA_PATH, 'processed/trees/ok_trees.gpkg')
BEECHES_LAYER='ok_trees_20_40_m'

written_files=[]


# Read data

In [ ]:
beeches=gpd.read_file(BEECHES_POLYGONS, layer=BEECHES_LAYER)

beeches.drop(columns=['Comm', 'essence', 'diam_tronc', 'nb_tronc', 'hauteur', 'verticalit', 'diametre_c', 'mortalite_',
                'transparen', 'masse_foli', 'etat_tronc', 'etat_sanit', 'environnem',
                'microtopog', 'pente', 'remarque', 'date_leve', 'responsabl',
                'date_creat', 'vegetation', 'class_san5', 'class_san3', 'r_couronne', 'zone'], inplace=True)

In [ ]:
# tiles_list_north=glob(NORTH_NDVI)
# tiles_list_south=glob(SOUTH_NDVI)
aoi_gt=gpd.read_file(AOI_GT)

In [ ]:
north_chm=fct_misc.polygonize_binary_raster(NORTH_MASK)
south_chm=fct_misc.polygonize_binary_raster(SOUTH_MASK)

chm=pd.concat([north_chm, south_chm])
del north_chm, south_chm

# Treat data

In [ ]:
# fct_misc.test_crs(beeches.crs, chm.crs)
# correct_high_beeches=gpd.overlay(beeches, chm)
# correct_high_beeches.drop(columns=['class'], inplace=True)

correct_high_beeches=beeches.copy()

In [ ]:
aoi_gt.head()

In [ ]:
rgb_pathes=[]
ndvi_pathes=[]

for tile_name in aoi_gt['NAME'].values:
    if tile_name.startswith('257'):
        rgb_pathes.append(os.path.join(
                            DATA_PATH,
                            'initial/True_ortho/Tiled_North', 'North_ortho_JUHE_LV95_NF02_3cm_' + tile_name + '.tif'
        ))
        ndvi_pathes.append(os.path.join(
                            DATA_PATH, 
                            'processed/NDVI', 'North_NDVI_' + tile_name + '.tif'
        ))

    elif tile_name.startswith('258'):
        rgb_pathes.append(os.path.join(
                            DATA_PATH,
                            'initial/True_ortho/Tiled_South', 'South_ortho_JUHE_LV95_NF02_3cm_' + tile_name + '.tif'
        ))
        ndvi_pathes.append(os.path.join(
                            DATA_PATH, 
                            'processed/NDVI', 'South_NDVI_' + tile_name + '.tif'
        ))


In [ ]:
aoi_gt['path_RGB']=rgb_pathes
aoi_gt['path_NDVI']=ndvi_pathes


In [ ]:
clipped_beeches=fct_misc.clip_labels(correct_high_beeches, aoi_gt)

In [ ]:
for health_class in clipped_beeches['etat_sanitaire'].unique():
    
    print(f"There are {clipped_beeches[clipped_beeches['etat_sanitaire']==health_class].shape[0]} beeches with the health status '{health_class}'")


In [ ]:
clipped_beeches.crs



In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Warning 1: The definition of projected CRS EPSG:2056 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.")

In [ ]:
clipped_beeches=clipped_beeches[~clipped_beeches.is_empty]


In [ ]:
beeches_stats=pd.DataFrame()
BANDS={1: 'rouge', 2: 'vert', 3: 'bleu', 4: 'proche IR'}
calculated_stats=['min', 'max', 'mean', 'median', 'std']

for beech in clipped_beeches.itertuples():
    for band_num in BANDS.keys():
        stats_rgb=zonal_stats(beech.geometry, beech.path_RGB, stats=calculated_stats,
        band=band_num, nodata=9999)

        stats_dict_rgb=stats_rgb[0]
        stats_dict_rgb['band']=BANDS[band_num]
        stats_dict_rgb['health_status']=beech.etat_sanitaire

        beeches_stats=pd.concat([beeches_stats, pd.DataFrame(stats_dict_rgb, index=[0])], ignore_index=True)
    
    stats_ndvi=zonal_stats(beech.geometry, beech.path_NDVI, stats=calculated_stats,
        band=1, nodata=99999)

    stats_dict_ndvi=stats_ndvi[0]
    stats_dict_ndvi['band']='ndvi'
    stats_dict_ndvi['health_status']=beech.etat_sanitaire

    beeches_stats=pd.concat([beeches_stats, pd.DataFrame(stats_dict_ndvi, index=[0])], ignore_index=True)


In [ ]:
beeches_stats.loc[beeches_stats['health_status']=='sain', 'health_status']='1. sain'
beeches_stats.loc[beeches_stats['health_status']=='malade', 'health_status']='2. malade'
beeches_stats.loc[beeches_stats['health_status']=='mort', 'health_status']='3. mort'


In [ ]:
beeches_stats.head(10)

In [ ]:
table_path=fct_misc.ensure_dir_exists(os.path.join(DATA_PATH, 'final/tables'))
im_path=fct_misc.ensure_dir_exists(os.path.join(DATA_PATH, 'final/images'))

## Make boxplots

In [ ]:
for band in beeches_stats['band'].unique():

    band_stats=beeches_stats[beeches_stats['band']==band]

    bxplt_beeches=band_stats[calculated_stats + ['health_status']].plot.box(
                                by='health_status',
                                title=f'Distribution des statistiques sur les hêtres pour la bande {band}',
                                figsize=(18, 5),
                                grid=True,
    )

    fig=bxplt_beeches[0].get_figure()
    filepath=os.path.join(im_path, f'boxplot_stats_band_{band}.jpg')
    fig.savefig(filepath, bbox_inches='tight')
    written_files.append(filepath)


## Make PCA

In [ ]:
for band in beeches_stats['band'].unique():
    features = calculated_stats
    to_describe='health_status'
    band_stats=beeches_stats[beeches_stats['band']==band]

    written_files_pca_pixels=pca_procedure(band_stats, features, to_describe,
                            table_path, im_path, 
                            file_prefix=f'PCA_beeches_{band}_band',
                            title_graph=f'PCA des hêtres en fonction de leur état de santé sur la bande {band}')

    written_files.extend(written_files_pca_pixels)